In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\khang\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Log in to your W&B account
import wandb
import psutil
import time

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
import psutil
import wandb

def log_cpu_ram_usage():
    # Đo CPU và RAM
    cpu_percent = psutil.cpu_percent()
    ram_percent = psutil.virtual_memory().percent
    ram_used = psutil.virtual_memory().used / (1024 ** 3)  # Convert to GB

    # Log vào WandB
    wandb.log({
        "CPU_Usage (%)": cpu_percent,
        "RAM_Usage (%)": ram_percent,
        "RAM_Used (GB)": ram_used
    })


In [ ]:
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetUtilizationRates, nvmlDeviceGetMemoryInfo

def log_gpu_usage():
    try:
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)  # Assuming a single GPU
        gpu_utilization = nvmlDeviceGetUtilizationRates(handle)
        memory_info = nvmlDeviceGetMemoryInfo(handle)

        # Log vào WandB
        wandb.log({
            "GPU_Usage (%)": gpu_utilization.gpu,
            "GPU_Memory_Used (GB)": memory_info.used / (1024 ** 3),  # Convert to GB
            "GPU_Memory_Total (GB)": memory_info.total / (1024 ** 3)
        })
    except Exception as e:
        print(f"Error logging GPU usage: {e}")


In [ ]:
# Load the datasets
data1 = pd.read_csv('ctgan_sample/ctgan_90_20e_iot20_sample.csv')
data2 = pd.read_csv('iotid20.csv')

# Assuming the last column is the target and all others are features
data1.replace([np.inf, -np.inf], np.nan, inplace=True)
data1.dropna(inplace=True)

data2.replace([np.inf, -np.inf], np.nan, inplace=True)
data2.dropna(inplace=True)


In [ ]:
df1 = pd.DataFrame(data1)

# Phân loại các cột theo kiểu dữ liệu
#numerical_cols = df1.select_dtypes(include=['int64', 'float64']).columns
#categorical_cols = df1.select_dtypes(include=['object']).columns

# Tiền xử lý cột số bằng StandardScaler
#scaler = StandardScaler()
#df1[numerical_cols] = scaler.fit_transform(df1[numerical_cols])

# Tiền xử lý cột phân loại bằng LabelEncoder
#label_encoders = {}
#for col in categorical_cols:
 ##   df1[col] = le.fit_transform(df1[col])
  #  label_encoders[col] = le

In [ ]:
df2 = pd.DataFrame(data2)

# Phân loại các cột theo kiểu dữ liệu
#numerical_cols = df2.select_dtypes(include=['int64', 'float64']).columns
#categorical_cols = df2.select_dtypes(include=['object']).columns

# Tiền xử lý cột số bằng StandardScaler
##df2[numerical_cols] = scaler.fit_transform(df2[numerical_cols])

# Tiền xử lý cột phân loại bằng LabelEncoder
#label_encoders = {}
#for col in categorical_cols:
#    le = LabelEncoder()
#    df2[col] = le.fit_transform(df2[col])
#    label_encoders[col] = le

In [ ]:
scaler = StandardScaler()
label_encoder = LabelEncoder()

X1, y1 = df1.iloc[:, :-1], data1.iloc[:, -1]
X2, y2 = df2.iloc[:, :-1], data2.iloc[:, -1]

X1 = scaler.fit_transform(X1)
X2 = scaler.fit_transform(X2)
y1 = label_encoder.fit_transform(y1)
y2 = label_encoder.fit_transform(y2)
# Split dataset 1 into training and test sets
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

# Split dataset 2 into training and test sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Train MLP using the training set of the first dataset and test on the second dataset's test set


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Train on Dataset 1
mlp.fit(X1_train, y1_train)
wandb.init(
    project="ct_dim_usage"
    config={
        "model": "MLPClassifier",
        "hidden_layers": (100,),
        "max_iter": 500,
        "dataset": "IoTID20",
        "random_state": 42
    }
)
log_cpu_ram_usage()
log_gpu_usage()

# Test on the test set of Dataset 2
y2_pred = mlp.predict(X2_test)
accuracy_1 = accuracy_score(y2_test, y2_pred)
print(f"Accuracy using Dataset 1 for training and Dataset 2 for testing: {accuracy_1:.2f}")

# Tính toán F1 Score
f1_1 = f1_score(y2_test, y2_pred, average='weighted')  # average='weighted' để tính trọng số cho mỗi lớp
print(f"F1 Score: {f1_1:.2f}")

# Tính toán Precision
precision_1 = precision_score(y2_test, y2_pred, average='weighted')
print(f"Precision: {precision_1:.2f}")

# Tính toán Recall
recall_1 = recall_score(y2_test, y2_pred, average='weighted')
print(f"Recall: {recall_1:.2f}")

wandb.log({
    "Accuracy": accuracy_1,
    "F1_Score": f1_1,
    "Precision": precision_1,
    "Recall": recall_1
})

Accuracy using Dataset 1 for training and Dataset 2 for testing: 0.74
F1 Score: 0.72
Precision: 0.73
Recall: 0.74


In [ ]:
#from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Dự đoán trên toàn bộ Dataset 2 (test set)
#y2_full_pred = mlp.predict(X2_test)

# Tính toán accuracy
#accuracy_2 = accuracy_score(y2_test, y2_full_pred)
#print(f"Accuracy using the whole Dataset 2 for both training and testing: {accuracy_2:.2f}")

# Tính toán F1 Score
#f1_2 = f1_score(y2_test, y2_full_pred, average='weighted')  # average='weighted' để tính trọng số cho mỗi lớp
#print(f"F1 Score: {f1_2:.2f}")

# Tính toán Precision
#precision_2 = precision_score(y2_test, y2_full_pred, average='weighted')
#print(f"Precision: {precision_2:.2f}")

# Tính toán Recall
#recall_2 = recall_score(y2_test, y2_full_pred, average='weighted')
#print(f"Recall: {recall_2:.2f}")


# Compare accuracies


In [ ]:
#if accuracy_2 > accuracy_1:
 #   print("Training on the full second dataset gives better accuracy.")
#else:
#    print("Training on the first dataset gives better accuracy on the second test set.")